In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import keras
import random
from tensorflow.keras.callbacks import EarlyStopping

# ***Movie Recommender System***

## Datasets Preprocessing


In [2]:
movies = pd.read_csv("movies.csv")
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [3]:
ratings = pd.read_csv("ratings.csv")
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [4]:
ratings.duplicated().sum()

np.int64(0)

In [5]:
ratings.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


In [6]:
# ratings.drop(columns=["timestamp"], inplace=True)
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [7]:
movie_ratings = pd.merge(movies, ratings, on='movieId')
movie_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,2000-07-30 18:45:03
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1996-11-08 06:36:02
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,2005-01-25 06:52:26
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,2017-11-13 12:59:30
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,2011-05-18 05:28:03


In [8]:
# Time-aware per-user split: train/val/test
ratings_sorted = ratings.sort_values(['userId', 'timestamp'])

def split_user(g, val_ratio=0.1, test_ratio=0.1):
    n = len(g)
    test_n = max(1, int(n * test_ratio))
    val_n  = max(1, int(n * val_ratio))
    return g.iloc[:n - val_n - test_n], g.iloc[n - val_n - test_n:n - test_n], g.iloc[n - test_n:]

train_parts, val_parts, test_parts = [], [], []
for uid, g in ratings_sorted.groupby('userId', sort=False):
    tr, va, te = split_user(g)
    train_parts.append(tr); val_parts.append(va); test_parts.append(te)

train_ratings = pd.concat(train_parts).reset_index(drop=True)
val_ratings   = pd.concat(val_parts).reset_index(drop=True)
test_ratings  = pd.concat(test_parts).reset_index(drop=True)

print(
    f"Train: {len(train_ratings)}  Val: {len(val_ratings)}  Test: {len(test_ratings)}  "
    f"Users: {ratings['userId'].nunique()}  Items: {ratings['movieId'].nunique()}"
)

Train: 81200  Val: 9818  Test: 9818  Users: 610  Items: 9724


In the above step, I’m splitting the data for each user based on time instead of randomly.
This means that a user’s earlier ratings go into the training set, and their more recent ratings go into validation and test sets. This approach makes the model more realistic because, in the real world, we always predict future preferences based on past behavior.
A random split would mix up old and new ratings, letting the model “see the future,” which leads to unfairly good results.
Using a time-aware split ensures the evaluation reflects how the recommender would actually perform in practice.

## **Collaborative filtering using matrix factorization (SVD)**

### Creating user-item interaction matrix

In [9]:
user_movie_matrix = train_ratings.pivot_table(
    index='userId', columns='movieId', values='rating'
)
user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,189043,189111,189333,193565,193567,193571,193573,193579,193581,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Computing mean centered ratings

In [10]:
R = user_movie_matrix.values
mask = ~np.isnan(R)

# Per-user means over observed entries
user_means = np.divide(
    np.nansum(R, axis=1),
    np.maximum(mask.sum(axis=1), 1),
    out=np.zeros(R.shape[0], dtype=float),
    where=True
)

# Center only observed entries; keep missing as 0 (safe for sparse SVD)
R_centered = np.where(mask, R - user_means[:, None], 0.0)

# Sparse representation for truncated SVD
R_centered_sparse = csr_matrix(R_centered)

In the above step, I’m centering each user’s ratings around their personal average before applying SVD.
This is important because some users give higher or lower ratings on average — for example, one user might rate everything around 4, while another stays near 2.
By subtracting each user’s mean rating, we remove these personal biases and focus on how a user’s preferences differ across movies.
This makes the patterns between users and movies clearer for SVD to learn.
After the predictions are made, the user means are added back to bring ratings back to the original scale.

### Applying SVD (Single Value Decomposition)

This part breaks down the user-movie rating matrix into smaller matrices to discover hidden patterns—like a user's taste or a movie's type. By multiplying these parts back together, we can predict ratings for movies the user hasn’t rated yet.

In [11]:
k = 50

#factorizing the matrix into three matrices U,sigma and Vt
U, sigma, Vt = svds(R_centered_sparse, k=k)
sigma_diag_matrix = np.diag(sigma)
R_hat_centered = np.dot(np.dot(U, sigma_diag_matrix), Vt)
R_hat = R_hat_centered + user_means[:, np.newaxis]

#Predicted Ratings as DataFrame
predicted_ratings = pd.DataFrame(R_hat, index=user_movie_matrix.index, columns=user_movie_matrix.columns)

In [12]:
predicted_ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,189043,189111,189333,193565,193567,193571,193573,193579,193581,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.428785,4.368396,4.372917,4.388743,4.379505,4.238432,4.291834,4.387466,4.395247,4.325181,...,4.371933,4.371209,4.370923,4.371578,4.372561,4.370594,4.370594,4.371578,4.370594,4.369832
2,3.943029,3.927557,3.929468,3.946141,3.948149,3.915425,3.931026,3.941897,3.946375,3.956060,...,3.940654,3.940164,3.939989,3.940057,3.940150,3.939965,3.939965,3.940057,3.939965,3.939532
3,2.778665,2.803412,2.744299,2.794679,2.777914,2.767335,2.863426,2.812359,2.780428,2.781010,...,2.787275,2.787728,2.787920,2.787860,2.787829,2.787891,2.787891,2.787860,2.787891,2.788384
4,3.866750,3.414535,3.683462,3.583547,3.706003,3.366243,3.739514,3.544353,3.653224,3.663868,...,3.618739,3.615892,3.615185,3.617189,3.620810,3.613568,3.613568,3.617189,3.613568,3.607711
5,3.718419,3.563822,3.581872,3.554485,3.556988,3.670588,3.575827,3.577897,3.565454,3.534872,...,3.580099,3.582525,3.583318,3.583590,3.584003,3.583177,3.583177,3.583590,3.583177,3.584080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,3.477102,3.554181,3.779095,3.743348,3.793086,3.303430,2.352129,3.616301,3.693167,4.024829,...,3.737892,3.732485,3.730677,3.733244,3.737372,3.729116,3.729116,3.733244,3.729116,3.734911
607,3.811683,3.981321,3.918292,3.834456,3.896117,3.942321,3.762562,3.845072,3.827192,3.730537,...,3.846294,3.847335,3.847813,3.846947,3.845763,3.848132,3.848132,3.846947,3.848132,3.848425
608,2.428302,1.855047,1.982469,3.048545,3.060017,3.291807,2.957513,3.055274,3.012501,4.111391,...,3.058424,3.062538,3.063787,3.064977,3.066698,3.063257,3.063257,3.064977,3.063257,3.050359


### Generating recommendations with explanation

In [13]:
def get_top_n_recommendations(user_id, n=5):
    #only users present in train/user_movie_matrix
    if user_id not in predicted_ratings.index:
        return f"User {user_id} not in training set. (Cold-start user)"

    user_row = predicted_ratings.loc[user_id]

    #Excluding entries that were observed in train
    already_rated = user_movie_matrix.loc[user_id].dropna().index
    recommendations = user_row.drop(index=already_rated).sort_values(ascending=False).head(n)

    for movie_id in recommendations.index:
        movie_title = movies.loc[movies['movieId'] == movie_id, 'title'].values[0]
        pred_rating = float(user_row[movie_id])

        actual_ratings = ratings[ratings['movieId'] == movie_id]
        avg_rating = actual_ratings['rating'].mean()
        total_ratings = actual_ratings.shape[0]

        print(f'Recommended: "{movie_title}" | Predicted Rating: {pred_rating:.2f}★')
        print(f'  Reason: Based on patterns learned from your past ratings and similar users/items.')
        if total_ratings > 0:
            print(f'  (Info: {total_ratings} total ratings; average {avg_rating:.2f}★.)\n')
        else:
            print(f'  (Info: No existing ratings available for this movie.)\n')


In [14]:
user_id = 1
print(f"Top 5 recommendations for User {user_id}:")
print(get_top_n_recommendations(user_id))

Top 5 recommendations for User 1:
Recommended: "Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)" | Predicted Rating: 4.71★
  Reason: Based on patterns learned from your past ratings and similar users/items.
  (Info: 97 total ratings; average 4.27★.)

Recommended: "2001: A Space Odyssey (1968)" | Predicted Rating: 4.66★
  Reason: Based on patterns learned from your past ratings and similar users/items.
  (Info: 109 total ratings; average 3.89★.)

Recommended: "Blade Runner (1982)" | Predicted Rating: 4.65★
  Reason: Based on patterns learned from your past ratings and similar users/items.
  (Info: 124 total ratings; average 4.10★.)

Recommended: "Godfather, The (1972)" | Predicted Rating: 4.62★
  Reason: Based on patterns learned from your past ratings and similar users/items.
  (Info: 192 total ratings; average 4.29★.)

Recommended: "Austin Powers: The Spy Who Shagged Me (1999)" | Predicted Rating: 4.59★
  Reason: Based on patterns learned from your past rat

### Evaluataing performance of model

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

actual, predicted = [], []

for row in test_ratings.itertuples(index=False):
    user = row.userId
    movie = row.movieId
    true_rating = row.rating

    #only score if both user and item exist in the trained matrix
    if (user in predicted_ratings.index) and (movie in predicted_ratings.columns):
        pred_rating = predicted_ratings.loc[user, movie]
        #skip NaN predicted rating for unseen items
        if not np.isnan(pred_rating):
            actual.append(true_rating)
            predicted.append(pred_rating)

svd_mse = mean_squared_error(actual, predicted) if actual else np.nan
svd_rmse = np.sqrt(svd_mse) if actual else np.nan
svd_mae = mean_absolute_error(actual, predicted) if actual else np.nan

print("SVD Evaluation on TEST set:")
print(f"Pairs evaluated: {len(actual)} / {len(test_ratings)}")
print(f"RMSE: {svd_rmse:.4f}")
print(f"MAE : {svd_mae:.4f}")

SVD Evaluation on TEST set:
Pairs evaluated: 8886 / 9818
RMSE: 0.9628
MAE : 0.7459


The final results show an RMSE of about 0.96 and an MAE of around 0.75 on a 1–5 rating scale.
This means the model’s predictions are, on average, less than one rating point away from the true value.
While this is not state-of-the-art accuracy, it’s a reasonable outcome for a basic matrix-factorization model without heavy tuning or additional bias correction.
The results indicate that the model captures general rating trends but could still be improved with more advanced techniques or parameter optimization.
Overall, the evaluation shows that the approach works correctly and produces meaningful, if not perfect, recommendations.

##**Deep Neural Network**

### Pre-processing genres column

In [16]:
movies['genres'] = movies['genres'].str.split('|')

#one hot encoding genres
genre_encoder = MultiLabelBinarizer()
genre_features = genre_encoder.fit_transform(movies['genres'])

genre_df = pd.DataFrame(genre_features, columns=genre_encoder.classes_)
genre_df['movieId'] = movies['movieId']

movie_ratings = movie_ratings.merge(genre_df, on='movieId', how='left')

In [17]:
movie_ratings.head()

,movieId,title,genres,userId,rating,timestamp,(no genres listed),Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,2000-07-30 18:45:03,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1996-11-08 06:36:02,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,2005-01-25 06:52:26,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,2017-11-13 12:59:30,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,2011-05-18 05:28:03,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


### Adding tags

In [19]:
tags = pd.read_csv("tags.csv")
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [20]:
tag_strings = tags.groupby('movieId')['tag'].apply(lambda tags: ' '.join(tags)).reset_index()

movie_ratings = movie_ratings.merge(tag_strings, on='movieId', how='left')
movie_ratings['tag'] = movie_ratings['tag'].fillna('')

In [21]:
movie_ratings.head()

,movieId,title,genres,userId,rating,timestamp,(no genres listed),Action,Adventure,Animation,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,2000-07-30 18:45:03,0,0,1,1,...,0,0,0,0,0,0,0,0,0,pixar pixar fun
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1996-11-08 06:36:02,0,0,1,1,...,0,0,0,0,0,0,0,0,0,pixar pixar fun
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,2005-01-25 06:52:26,0,0,1,1,...,0,0,0,0,0,0,0,0,0,pixar pixar fun
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,2017-11-13 12:59:30,0,0,1,1,...,0,0,0,0,0,0,0,0,0,pixar pixar fun
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,2011-05-18 05:28:03,0,0,1,1,...,0,0,0,0,0,0,0,0,0,pixar pixar fun


In [22]:
tfidf = TfidfVectorizer(max_features=100)  #limit to top 100 features
tag_features = tfidf.fit_transform(movie_ratings['tag']).toarray()
tag_df = pd.DataFrame(tag_features, columns=tfidf.get_feature_names_out())

movie_ratings = pd.concat([movie_ratings.reset_index(drop=True), tag_df.reset_index(drop=True)], axis=1)

In [23]:
movie_ratings.head()

,movieId,title,genres,userId,rating,timestamp,(no genres listed),Action,Adventure,Animation,...,thriller,time,timeline,top,travel,twist,violence,violent,visually,war
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,2000-07-30 18:45:03,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1996-11-08 06:36:02,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,2005-01-25 06:52:26,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,2017-11-13 12:59:30,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,2011-05-18 05:28:03,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Encoding data

In [24]:
#encoding the user/movie IDs into proper index ranges for embeddings
user_encoder = LabelEncoder()
movie_ratings['user'] = user_encoder.fit_transform(movie_ratings['userId'].values)
n_users = movie_ratings['user'].nunique()


movie_encoder = LabelEncoder()
movie_ratings['movie'] = movie_encoder.fit_transform(movie_ratings['movieId'].values)
n_movies = movie_ratings['movie'].nunique()

print("Total users: ", n_users)
print("Total movies: ", n_movies)

Total users:  610
Total movies:  9724


In [25]:
movie_ratings['rating'] = movie_ratings['rating'].values.astype(np.float32)
min_rating = min(movie_ratings['rating'])
max_rating = max(movie_ratings['rating'])

print("Minimum rating: ", min_rating)
print("Maximum rating: ", max_rating)

Minimum rating:  0.5
Maximum rating:  5.0


### Splitting data for training and testing

In [26]:
X_user = movie_ratings['user'].values
X_movie = movie_ratings['movie'].values
X_genres = movie_ratings[genre_encoder.classes_].values
X_tags = movie_ratings[tfidf.get_feature_names_out()].values

X_content = np.hstack([X_genres, X_tags])
# scaler = MinMaxScaler()
# X_content = scaler.fit_transform(X_content)

In [27]:
y = movie_ratings['rating'].values

#split into train/test
X_user_train, X_user_test, X_movie_train, X_movie_test, X_content_train_raw, X_content_test_raw, y_train, y_test = train_test_split(
    X_user, X_movie, X_content, y, test_size=0.2, random_state=42
)

#split train into train/val
X_user_train, X_user_val, X_movie_train, X_movie_val, X_content_train_raw, X_content_val_raw, y_train, y_val = train_test_split(
    X_user_train, X_movie_train, X_content_train_raw, y_train, test_size=0.2, random_state=42
)

#scale content features using train only
scaler = MinMaxScaler()
X_content_train = scaler.fit_transform(X_content_train_raw)
X_content_val   = scaler.transform(X_content_val_raw)
X_content_test  = scaler.transform(X_content_test_raw)


In [28]:
#normalize targets to [0,1] for sigmoid
y_train = (y_train - min_rating) / (max_rating - min_rating)
y_val   = (y_val   - min_rating) / (max_rating - min_rating)
y_test  = (y_test  - min_rating) / (max_rating - min_rating)

### Creating embeddings

In [29]:
#initializing number of dimensions of our embedding layer (hyper-parameter)
n_dim = 64

In [30]:
#creating input layers
user = tf.keras.layers.Input(shape=(1,))
movie = tf.keras.layers.Input(shape=(1,))
content = tf.keras.Input(shape=(X_content.shape[1],))

In [31]:
#embedding layers created to turn ids into a dense vector of numbers for user and movies separately
user_emb = keras.layers.Embedding(n_users, n_dim, embeddings_initializer='he_normal',
                           embeddings_regularizer=tf.keras.regularizers.l2(1e-6), name='user-embeddings')(user)
user_emb = tf.keras.layers.Reshape((n_dim,))(user_emb)

movie_emb = keras.layers.Embedding(n_movies, n_dim, embeddings_initializer='he_normal',
                           embeddings_regularizer=tf.keras.regularizers.l2(1e-6), name='movie-embeddings')(movie)
movie_emb = tf.keras.layers.Reshape((n_dim,))(movie_emb)

These embedding layers capture the latent (hidden) properties of each user and movie that explain their preferences and characteristics by learning patterns in the data based purely on what users rate highly or poorly.

In [32]:
#concatenate embeddings with content features
x = tf.keras.layers.Concatenate()([user_emb, movie_emb, content])
x = tf.keras.layers.Dropout(0.30)(x)

Dropout layer added to zero-out some embedding vector values to ensure the model generalizes well and doesn't overfit to the training data.

### Creating our neural network

Here we have built a deep learning model that learns from user IDs and movie IDs by turning them into special number vectors (embeddings). These vectors help the model understand hidden features, like what kind of movies a user likes. The model connects these embeddings through layers to learn patterns and predict how much a user will like a movie.

In [33]:
#dense layers
x = tf.keras.layers.Dense(32, kernel_initializer='he_normal', activation='relu')(x)
x = tf.keras.layers.Dropout(0.30)(x)
x = tf.keras.layers.Dense(16, kernel_initializer='he_normal', activation='relu')(x)
x = tf.keras.layers.Dropout(0.30)(x)

In [34]:
#output layer for regression
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

Using sigmoid activation function since our ratings(y) are normalized to a scale of 0-1

In [35]:
model = tf.keras.models.Model(inputs=[user, movie, content], outputs=x)

In [36]:
#compiling the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [37]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user-embeddings     │ (None, 1, 64)     │     39,040 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie-embeddings    │ (None, 1, 64)     │    622,336 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 64)        │          0 │ user-embeddings[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 64)        │          0 │ movie-embeddings… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 120)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 248)       │          0 │ reshape[0][0],    │
│ (Concatenate)       │                   │            │ reshape_1[0][0],  │
│                     │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 248)       │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │      7,968 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 32)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │        528 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 16)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         17 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 669,889 (2.56 MB)

 Trainable params: 669,889 (2.56 MB)

 Non-trainable params: 0 (0.00 B)

### Training DNN on data

In [38]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.75, patience=3, min_lr=1e-6, verbose=1
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

trained_model = model.fit(
    x=[X_user_train, X_movie_train, X_content_train], y=y_train,
    validation_data=([X_user_val, X_movie_val, X_content_val], y_val),
    epochs=50,
    batch_size=2048,
    shuffle=True,
    callbacks=[reduce_lr, early_stop],
    verbose=1
)

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 133ms/step - loss: 0.0675 - mae: 0.2136 - val_loss: 0.0483 - val_mae: 0.1739 - learning_rate: 0.0010
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0514 - mae: 0.1781 - val_loss: 0.0413 - val_mae: 0.1570 - learning_rate: 0.0010
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0437 - mae: 0.1620 - val_loss: 0.0396 - val_mae: 0.1518 - learning_rate: 0.0010
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0400 - mae: 0.1531 - val_loss: 0.0391 - val_mae: 0.1510 - learning_rate: 0.0010
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0382 - mae: 0.1500 - val_loss: 0.0386 - val_mae: 0.1502 - learning_rate: 0.0010
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0373 - mae: 0.1478 - val_loss: 0.0383 - val_mae: 0.1496 - learning_rate: 0.0010
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0357 - mae: 0.1449 - val_loss: 0.0380 - val_mae: 0.1490 - learning_rate: 0.0010
Epoch 8/50
32/32 ━

ReduceLROnPlateau lowers the learning rate automatically if the model stops improving. It helps the model fine-tune better by taking smaller steps when it's stuck, which can lead to a better final result.



### Generating recommendation with explanation

In [40]:
def get_dnn_recommendations(user_id, n=5):
    movie_ids = movies['movieId'].unique()
    seen_movies = ratings[ratings['userId'] == user_id]['movieId'].tolist()

    known_movie_ids = set(movie_encoder.classes_)
    unseen_movies = [m for m in movie_ids if m not in seen_movies and m in known_movie_ids]
    user_array = np.array([user_encoder.transform([user_id])[0]] * len(unseen_movies))
    movie_array = movie_encoder.transform(unseen_movies)

    content_cols = genre_encoder.classes_.tolist() + list(tfidf.get_feature_names_out())
    unseen_content_df = movie_ratings.drop_duplicates(subset='movieId')
    unseen_content_df = unseen_content_df.set_index('movieId').loc[unseen_movies]
    content_array = unseen_content_df[content_cols].values

    #predicting ratings for unseen movies
    raw_prediction = model.predict([user_array, movie_array, content_array], verbose=0).flatten()
    predicted_ratings = raw_prediction * (max_rating - min_rating) + min_rating

    rec_df = pd.DataFrame({
        'movieId': unseen_movies,
        'predicted_rating': predicted_ratings
    })

    top_recommendations = rec_df.sort_values(by='predicted_rating', ascending=False).head(n)
    top_recommendations = top_recommendations.merge(movies[['movieId', 'title']], on='movieId', how='left')

    movie_embedding_layer = model.get_layer('movie-embeddings')
    movie_embeddings = movie_embedding_layer.get_weights()[0]

    print(f"\nTop {n} DNN recommendations for User {user_id}:\n")
    for idx, row in top_recommendations.iterrows():
        movie_id = row['movieId']
        movie_title = row['title']
        pred_rating = row['predicted_rating']

        #finding similar movies
        if movie_id in movie_encoder.classes_:
            target_idx = movie_encoder.transform([movie_id])[0]
            target_embedding = movie_embeddings[target_idx].reshape(1, -1)
            similarities = cosine_similarity(target_embedding, movie_embeddings)[0]

            #exclude self
            similar_movie_indices = similarities.argsort()[-6:-1][::-1]
            similar_titles = movies.iloc[similar_movie_indices]['title'].tolist()

            print(f'Recommended: "{movie_title}" | Predicted Rating: {pred_rating:.2f}★')
            print(f'  Reason: Based on similarity to movies you liked: {similar_titles}\n')
        else:
            print(f'Recommended: "{movie_title}" | Predicted Rating: {pred_rating:.2f}★')
            print(f'  Reason: Based on patterns learned from your rating history.\n')


In this project, since we are only using movies that were part of the dataset and encoded during training, the else block is unlikely to be triggered. However, we have kept it as a safety check in case new movies are added that weren't part of the original embeddings.

In [41]:
user_id = 1
print(get_dnn_recommendations(user_id, n=5))


Top 5 DNN recommendations for User 1:

Recommended: "Persuasion (1995)" | Predicted Rating: 4.81★
  Reason: Based on similarity to movies you liked: ['RKO 281 (1999)', 'Tenchi Muyô! In Love (1996)', 'Dragons: Gift of the Night Fury (2011)', 'Hoffa (1992)', 'Good Morning, Vietnam (1987)']

Recommended: "Casino (1995)" | Predicted Rating: 4.62★
  Reason: Based on similarity to movies you liked: ['Monkey Trouble (1994)', 'Zebraman (2004)', 'Day After, The (1983)', 'Flyboys (2006)', 'Pillow Talk (1959)']

Recommended: "City of Lost Children, The (Cité des enfants perdus, La) (1995)" | Predicted Rating: 4.57★
  Reason: Based on similarity to movies you liked: ['Fury (1936)', 'Burrowers, The (2008)', 'Insidious: Chapter 2 (2013)', 'Dracula Untold (2014)', 'The Girls (1961)']

Recommended: "GoldenEye (1995)" | Predicted Rating: 4.56★
  Reason: Based on similarity to movies you liked: ['High Heels and Low Lifes (2001)', 'Bull Durham (1988)', 'Beauty and the Beast: The Enchanted Christmas (199

### Evaluating performance of model

In [42]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

#make predictions on the test set
y_pred = model.predict([X_user_test, X_movie_test, X_content_test], verbose=0).flatten()

#unnormalize predictions back to the original rating scale
y_pred_rescaled = y_pred * (max_rating - min_rating) + min_rating
y_true_rescaled = y_test * (max_rating - min_rating) + min_rating

#compute regression metrics
rmse = np.sqrt(mean_squared_error(y_true_rescaled, y_pred_rescaled))
mae = mean_absolute_error(y_true_rescaled, y_pred_rescaled)

print("DNN Model Evaluation on Test Set:")
print(f"RMSE: {rmse:.4f}")
print(f"MAE : {mae:.4f}")

DNN Model Evaluation on Test Set:
RMSE: 0.8752
MAE : 0.6726


In this step, I’m evaluating how close the model’s predicted ratings are to the actual user ratings.
RMSE (Root Mean Squared Error) and MAE (Mean Absolute Error) both measure prediction accuracy where lower values mean better performance.
My model achieved an RMSE of 0.88 and an MAE of 0.67, which are quite good for a basic deep learning recommender.
This means the model’s predictions are, on average, less than one rating point away from the true ratings.
Overall, the results show that the model has learned useful patterns and can predict user preferences fairly accurately.

In [43]:
def recall_at_k(model, X_user, X_movie, X_content, y_true, k=10, threshold=4.0):
    user_ids = np.unique(X_user)
    recalls = []

    for uid in user_ids:
        #filter this user's interactions in the test set
        mask = X_user == uid
        if mask.sum() == 0:
            continue

        #true high-rated items for this user
        true_high = set(np.where((y_true[mask] * (max_rating - min_rating) + min_rating) >= threshold)[0])
        if len(true_high) == 0:
            continue

        #predict scores
        preds = model.predict([X_user[mask], X_movie[mask], X_content[mask]], verbose=0).flatten()
        top_k = set(np.argsort(preds)[-k:])

        recall = len(true_high & top_k) / len(true_high)
        recalls.append(recall)

    return np.mean(recalls) if recalls else 0.0

recall10 = recall_at_k(model, X_user_test, X_movie_test, X_content_test, y_test, k=10)
print(f"Average Recall@10: {recall10:.4f}")

Average Recall@10: 0.6789


Here, I’m checking how well the model recommends the movies users actually liked.
Recall@10 measures the percentage of a user’s favorite movies (ratings ≥ 4) that appear in the top 10 recommendations.
A Recall@10 of 0.68 means that, on average, about 68% of the movies each user truly liked were correctly ranked among their top 10 suggestions.
This shows that the model is doing a good job at identifying and recommending movies that match each user’s interests.
While there’s still room for improvement, these results indicate strong recommendation quality for a baseline neural model.